# Export all visuals

This notebook embeds a Power BI report in an interactive frame and then attempts to export the data from every visual on every (non-hidden) page. Note that not every visual type supports data export.

The data is exported in CSV or Excel format. A folder is created for the output, then a sub-folder is created for each page and a file is created for each visual.

When used to produce CSV files, this can be used in combination with the Visual Studio Code "Compare Folders" Extension to very quickly compare every cell of data from every visual in the report. This could be useful to "regression test" whether query, semantic model or page design changes between versions have only had the desired effect, and have not leaked into other pages or visuals.

https://marketplace.visualstudio.com/items?itemName=moshfeu.compare-folders

The notebook embeds a live frame containing the target report. So if you need to first apply specific filters or use slicers, you can run the notebook cell-by-cell down to get the embedded report frame, then interact with it just like it was a Power BI web browser tab.  When you then run the remaining notebook cells, the output will reflect your filter or slicer changes. Any changes made will not be saved back to the Power BI web report definition.

To run this, you will need to obtain your group (workspace) id and report id, e.g. from the URL of your report.  The target report must be published in a workspace.

This project started from a Microsoft sample notebook - refer to that for more possibilities for automated report interactions:

https://learn.microsoft.com/en-us/javascript/api/overview/powerbi/powerbi-jupyter

In [ ]:
from powerbiclient.authentication import DeviceCodeLoginAuthentication
from powerbiclient import Report 
from io import StringIO
from ipywidgets import interact
import pandas
import pathlib
import shutil
import string
import time

### Authenticate to Power BI


#### The following authentication methods can be used to authenticate the user and get the access token:

- [Device flow authentication](https://msal-python.readthedocs.io/en/latest/#msal.PublicClientApplication.acquire_token_by_device_flow)
    
    Code snippet:
    ``` py
  from powerbiclient.authentication import DeviceCodeLoginAuthentication
  device_auth = DeviceCodeLoginAuthentication()
    ```
- [Interactive authentication](https://msal-python.readthedocs.io/en/latest/#msal.PublicClientApplication.acquire_token_interactive)
    
    Code snippet:
    ``` py
  from powerbiclient.authentication import InteractiveLoginAuthentication
  interactive_auth = InteractiveLoginAuthentication()    
    ```

Note: Either use auth object created or access token acquired above while creating the report instance


#### Run the cell below to obtain the auth object using the Device Code Authentication method
Note once you have completed the authentication, you can keep using it for some time without re-authenticating, e.g. to export data from other reports.

In [ ]:
# Create Auth object - see output for instructions to complete authentication
auth = DeviceCodeLoginAuthentication()

### Embed the Power BI report

#### TO DO: Replace the output_dir and output_format parameters with your own selections.
The directory provided for output_dir must be in python format e.g. forward-slashes between folders.  It must end with a closing forward-slash "/" after the last folder.
 
Note the output_dir provided will be deleted and recreated, so be careful!!!  Don't try this with: output_dir = "C:/" ...

In [ ]:
output_dir = "ENTER YOUR TARGET OUTPUT FOLDER (will be deleted and recreated)"
output_format = "CSV" # CSV or EXCEL

#### TO DO: Replace the group_id and report_id parameters with your own group ID and report ID (found in the report URL using Power BI Web UI).

In [ ]:
group_id = "ENTER YOUR GROUP ID"
report_id = "ENTER YOUR REPORT ID"

report = Report(group_id=group_id, report_id=report_id, auth=auth)


#### Set report loaded event handler

In [ ]:
def loaded_callback(event_details):
    print('Report is loaded')

report.on('loaded', loaded_callback)

#### Set report rendered event handler

In [ ]:
def rendered_callback(event_details):
    print('Report is rendered')

report.on('rendered', rendered_callback)

#### Load the report in the output cell

You can manually interact with the report (eg applying filters etc) before running the subsequent notebook cells to complete the export.

In [ ]:
# Render report
report

#### Change dimensions of embedded report

In [ ]:

# crude wait for report to render above (when all notebook cells are run)
time.sleep(15)
# Change dimensions of embedded report - adjust to suit your monitor
report.set_size(800, 1200) 

### Export data from all visuals of the embedded report

#### Delete and recreate the output directory

In [ ]:
dirpath = pathlib.Path(output_dir) 
if dirpath.exists() and dirpath.is_dir():
    shutil.rmtree(dirpath)

pathlib.Path(output_dir).mkdir(parents=True)

#### Loop over all the (non-hidden) pages, getting the visuals whose data is to be exported, then export in either CSV or Excel form

In [ ]:
# build a dictionary to later remove punctuation for folder and file names
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

#loop over the pages
report_pages=report.get_pages()
for report_page in report_pages:
    if report_page['visibility'] == 0:
        report_page_id = report_page['name']
        report_page_name = report_page['displayName']
        print(f"Page_id: {report_page_id}, Page_Name: {report_page_name}")
        # prepare the page name for use as a folder name - remove punctuation, truncate at 30 characters.
        report_page_name_folder = report_page_name.translate(remove_punctuation_map)[:30].strip()
        pathlib.Path(output_dir + report_page_name_folder).mkdir()
        report.set_active_page(report_page_id)
        
        # loop over the visuals on each page
        report_visuals=report.visuals_on_page(report_page_id)
        for report_visual in report_visuals:
            visual_id = report_visual['name']
            #some visuals will not have a Title property set, so wrap in a "try except" clause
            try:
                title = report_visual['title']
            except:
                title = None
            # prepare the title for use as a file name - remove punctuation, truncate at 30 characters.
            title_for_file_name = title.translate(remove_punctuation_map)[:30].strip()

            print(f"Page_id: {report_page_id}, Page_Name: {report_page_name}, Visual ID: {visual_id}, Visualization Type:{report_visual['type']}, Title: {title}")
            # I used "try except" because there is a long and changeable list of visual types that might or might not support this type of export (Summarised data)
            try:
                each_visual_df = pandas.read_csv(StringIO(report.export_visual_data(str(report_page_id), str(visual_id))), sep=",") 
            except:
                each_visual_df = None
            # print(each_visual_df)

            if each_visual_df is not None:
                output_filename = output_dir + report_page_name_folder + "/" + title_for_file_name + " " + visual_id 
                if output_format == "CSV":
                    print(f"Writing file: {output_filename}.csv")
                    each_visual_df.to_csv (output_filename + ".csv", index=False, header=True)
                else:
                    print(f"Writing file: {output_filename}.xlsx")
                    each_visual_df.to_excel (output_filename + ".xlsx", index=False)


#### Export finished

In [ ]:
print("Export finished!!")